# Parse TIFs

Use `deepometry.parse` module to transform .TIF files to NumPy arrays. In this example, .TIF files are stored at `/data/raw/` in subdirectories corresponding to the experiment, date, sample, replicate, and class labels. Filenames should contain a prefix "Ch" for specifying channels.


    /data/raw/
        Experiment 001/
            Day 1/
                Sample A/
                    Replicate 1/
                        Class A/
                            foo_Ch3.tif
                            foo_Ch4.tif
                            foo_Ch6.tif
                        Class B/
                            bar_Ch3.tif
                            bar_Ch4.tif
                            bar_Ch6.tif
        Experiment 002/
            Day 1/
                Sample A/
                    Replicate 1/
                        Class A/
                            foo_Ch3.tif
                            foo_Ch4.tif
                            foo_Ch6.tif
                        Class B/
                            bar_Ch3.tif
                            bar_Ch4.tif
                            bar_Ch6.tif
        ...

The images of selected channels of each object will be parsed into a numpy array, e.g. one cell - one numpy array that contains multiple channels (3D tensor). The arrays are stored at `/data/parsed` in subdirectories mirroring original data structure. Array filenames will have metadata prefixes, followed by a hex series.

    /data/parsed/
        Experiment 001/
            Day 1/
                Sample A/
                    Replicate 1/
                        Class A/
                            A__32e88e1ac3a8f44bf8f77371155553b9.npy
                            A__3dc56a0c446942aa0da170acfa922091.npy
                        Class B/
                            B__8068ef7dcddd89da4ca9740bd2ccb31e.npy
        Experiment 002/
            Day 1/
                Sample A/
                    Replicate 1/
                        Class A/
                            A__8348deaa70dfc95c46bd02984d28b873.npy
                        Class B/
                            B__c1ecbca7bd98c01c1d3293b64cd6739a.npy
                            B__c56cfb8e7e7121dd822e47c67d07e2d4.npy
        ...

# User's settings

In [ ]:
input_dir = '/data/raw/'
output_dir = '/data/parsed/'

channels = [1, 9, 12] # Cannot be None for TIFs

frame = 48

montage_size = 0 # Input number > 0 for stitching

# Executable

In [ ]:
import glob
import os.path

import bioformats
import javabridge

import deepometry.parse

# javabridge.start_vm(class_path=bioformats.JARS, max_heap_size="8G")

In [ ]:
all_subdirs = [x[0] for x in os.walk(input_dir)]

possible_labels = sorted(list(set([os.path.basename(i) for i in all_subdirs])))

# Book-keepers for all metadata
experiments = [i for i in possible_labels if 'experiment' in i.lower()]
days = [i for i in possible_labels if 'day' in i.lower()]
samples = [i for i in possible_labels if 'sample' in i.lower()]
replicates = [i for i in possible_labels if 'replicate' in i.lower()]
classes = [i for i in possible_labels if 'class' in i.lower()]

In [ ]:
print('Parsing... Please wait!')

for exp in experiments:
    for day in days:
        for sample in samples:
            for rep in replicates:
                for cl in classes:
                    folder_path = os.path.join(input_dir,exp,day,sample,rep,cl)

                    if os.path.exists(folder_path):
                        pathnames_tif = glob.glob(os.path.join(folder_path, '*.tif'))
                        pathnames_tiff = glob.glob(os.path.join(folder_path, '*.tiff'))
                        pathnames_cif = glob.glob(os.path.join(folder_path, '*.cif'))

                        for paths in [pathnames_tif, pathnames_tiff, pathnames_cif]:
                            if len(paths) > 0:
                                dest_dir = os.path.join(output_dir,exp,day,sample,rep,cl)

                                deepometry.parse.parse(
                                    paths=paths, 
                                    output_directory=dest_dir, 
                                    meta=exp + '_' + day + '_' + sample + '_' + rep + '_' + cl,                                        
                                    size=int(frame),
                                    channels=channels,
                                    montage_size=int(montage_size)
                                )
                                
print('Done.')